In [287]:
import torch
from torch import Tensor
import csv
import pandas as pd

def read_movies():
    # read csv with movies for budget and imdb_id
    columns_of_interest = ['budget', 'imdb_id', 'revenue', 'vote_average', 'directors']
    data = []
    with open('./data/movie_data_tmbd.csv', 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='|')
        for row in reader:
            extracted_row = {col: row[col] for col in columns_of_interest}
            data.append(extracted_row)

    movies_budget_df = pd.DataFrame(data)
    movies_budget_df = movies_budget_df.fillna({
        'budget': 0,
        'imdb_id': '',
        'title': '',
        'director': ''
    })

    # merge movie budget with id
    link_df = pd.read_csv("./data/small/links.csv")
    link_df['imdbId'] = link_df['imdbId'].apply(lambda x: f'tt0{int(x)}')

    movies_id_df = pd.merge(movies_budget_df, link_df, left_on='imdb_id', right_on='imdbId', how='inner')
    movies_id_df['budget'] = pd.to_numeric(movies_id_df['budget'])
    movies_id_df['revenue'] = pd.to_numeric(movies_id_df['revenue'])
    movies_id_df = movies_id_df[movies_id_df.budget != 0]
    movies_id_df = movies_id_df[movies_id_df.revenue != 0]

    movies_info_df = pd.read_csv("./data/small/movies.csv")
    movies_df = pd.merge(movies_id_df, movies_info_df, on="movieId", how="inner")

    ratings_df = pd.read_csv("./data/small/ratings.csv")
    #ratings_df = ratings_df.iloc[:ratings_df.shape[0]//10]
    ratings_df = ratings_df[ratings_df['movieId'].isin(movies_df['movieId'])]

    return movies_df, ratings_df

movies_df, ratings_df = read_movies()
#print(movies_df.head())
print(ratings_df.head())
print(len(movies_df))
print(len(ratings_df))

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
5       1       70     3.0  964982400
2429
55453


In [288]:
genres = movies_df['genres'].str.get_dummies('|')
movie_feat = torch.from_numpy(genres.values).to(torch.float)
movies_df['vote_average'].fillna(5.0, inplace=True)
vote_average = torch.from_numpy(movies_df['vote_average'].astype(float).values).to(torch.float).unsqueeze(-1)
movie_feat = torch.cat([movie_feat, vote_average], dim=1)
print(movie_feat.shape)

torch.Size([2429, 20])


/tmp/ipykernel_4981/676634563.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['vote_average'].fillna(5.0, inplace=True)


In [289]:
ratings_df['rating_classes'], unique_classes = pd.factorize(ratings_df['rating'])



In [290]:
import pandas as pd
from ast import literal_eval

# Check if conversion is needed and perform it safely
def safe_literal_eval(s):
    try:
        if isinstance(s, str):
            return literal_eval(s)
        else:
            return s  # assuming s is already the correct format
    except (ValueError, SyntaxError):
        return None

movies_df['directors'] = movies_df['directors'].apply(safe_literal_eval)

# Check the result after conversion
print(movies_df['directors'].head())

0    [{'credit_id': '52fe4640c3a36847f80f4a8b', 'de...
1    [{'credit_id': '52fe4652c3a36847f80f8afb', 'de...
2    [{'credit_id': '52fe466bc3a36847f80fdfb9', 'de...
3    [{'credit_id': '52fe466dc3a36847f80fe76d', 'de...
4    [{'credit_id': '57aa51cac3a368570a000786', 'de...
Name: directors, dtype: object


In [291]:
# Explode the DataFrame on the 'directors' column if the data is correctly formatted as lists of dictionaries
directors_expanded = movies_df.explode('directors').dropna()

# Check the first few rows to ensure explosion worked
print(directors_expanded['directors'].head())

0    {'credit_id': '52fe4640c3a36847f80f4a8b', 'dep...
1    {'credit_id': '52fe4652c3a36847f80f8afb', 'dep...
2    {'credit_id': '52fe466bc3a36847f80fdfb9', 'dep...
2    {'credit_id': '52fe466bc3a36847f80fdfb3', 'dep...
3    {'credit_id': '52fe466dc3a36847f80fe76d', 'dep...
Name: directors, dtype: object


In [292]:
# Normalize the directors' dictionary data into separate DataFrame columns
if not directors_expanded['directors'].isnull().all():  # Check if all values are not None
    directors_expanded[['director_id', 'director_name']] = directors_expanded['directors'].apply(lambda d: pd.Series({'id': d['id'], 'name': d['name']}) if pd.notna(d) else pd.Series({'id': None, 'name': None}))
else:
    print("No valid director data available.")

# Check if the new columns are filled correctly
print(directors_expanded[['director_id', 'director_name']].head())

   director_id  director_name
0         9181   Edward Zwick
1        59291  Alexandre Aja
2        53068     Ryan Fleck
2        53069     Anna Boden
3         3026     Rob Reiner


In [293]:
import pandas as pd
from ast import literal_eval

# Convert the 'vote_average' to float
directors_expanded['vote_average'] = pd.to_numeric(directors_expanded['vote_average'], errors='coerce')

# Check if the conversion was successful
print(directors_expanded[['director_name', 'vote_average']].head())

   director_name  vote_average
0   Edward Zwick           7.0
1  Alexandre Aja           5.3
2     Ryan Fleck           7.0
2     Anna Boden           7.0
3     Rob Reiner           7.7


In [294]:
# Group by director name and calculate mean ratings and count of movies
director_stats = directors_expanded.groupby('director_name').agg({
    'vote_average': ['mean', 'count']
}).reset_index()

# Flatten the multi-level column headers
director_stats.columns = ['director', 'average_rating', 'movie_counts']
director_stats = director_stats[['average_rating','movie_counts']]
# Display the resulting DataFrame
print(director_stats.shape)

(1229, 2)


In [295]:
director_stats = torch.from_numpy(director_stats.values).to(torch.float)
director_stats.shape

torch.Size([1229, 2])

In [296]:
directors_expanded = directors_expanded[['movieId', 'director_name', 'director_id']]
movies_extended_df = pd.merge(movies_df, directors_expanded, on='movieId', how='left')
movies_extended_df = movies_extended_df.dropna(subset=['director_name'])
print(len(movies_extended_df['director_name'].unique()))

1229


In [297]:
print(movies_extended_df['director_name'].unique())

['Edward Zwick' 'Alexandre Aja' 'Ryan Fleck' ... 'Oliver Parker'
 'Adam Elliot' 'Lance Mungia']


In [298]:
movies_extended_df['movieId'].unique()

array([82167, 79879, 80693, ...,  2275,  4141,  6289])

In [299]:
unique_director_id = movies_extended_df['director_id'].unique()
unique_director_id = pd.DataFrame(data={
    'director_id': unique_director_id,
    'mappedId': pd.RangeIndex(len(unique_director_id))
})
unique_movie_id = movies_extended_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedId': pd.RangeIndex(len(unique_movie_id))
})

director_director_id = pd.merge(movies_extended_df['director_id'], unique_director_id, on='director_id', how='left')
director_director_id = torch.from_numpy(director_director_id['mappedId'].values)
director_movie_id = pd.merge(movies_extended_df['movieId'], unique_movie_id, on='movieId', how='left')
director_movie_id = torch.from_numpy(director_movie_id['mappedId'].values)

edge_index_director_to_movie = torch.stack([director_director_id, director_movie_id], dim=0)
print(edge_index_director_to_movie.shape)

torch.Size([2, 2605])


In [300]:
# construct a compact representation of the data
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedId': pd.RangeIndex(len(unique_user_id)),
})
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedId': pd.RangeIndex(len(unique_movie_id)),
})

ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id, on='userId', how='left')
ratings_user_id = torch.from_numpy(ratings_user_id['mappedId'].values)
ratings_movie_id = pd.merge(ratings_df['movieId'], unique_movie_id, on='movieId', how='left')
ratings_movie_id = torch.from_numpy(ratings_movie_id['mappedId'].values)

edge_index_user_to_movie = torch.stack([ratings_user_id, ratings_movie_id], dim=0)

In [301]:
print(len(movies_extended_df['director_id'].unique()))
print(director_stats.shape)

1229
torch.Size([1229, 2])


In [302]:
torch.from_numpy(ratings_df['rating'].values).shape

torch.Size([55453])

In [303]:
edge_index_user_to_movie.shape

torch.Size([2, 55453])

In [304]:
print(len(ratings_df['movieId'].unique()))

2429


In [305]:

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
# save node indices
data['user'].node_id = torch.arange(len(unique_user_id))
data['movie'].node_id = torch.arange(len(unique_movie_id))
data['director'].node_id = torch.arange(len(unique_director_id))

# add node features
data['movie'].x = movie_feat
data['director'].x = director_stats
data['user', 'rates', 'movie'].edge_index = edge_index_user_to_movie
data['user', 'rates', 'movie'].edge_label = torch.from_numpy(ratings_df['rating'].values).to(torch.long) # TODO: this cuts off the .5 steps
data['director', 'directs', 'movie'].edge_index = edge_index_director_to_movie
data['director', 'directs', 'movie'].edge_label = torch.ones(edge_index_director_to_movie.shape[1]).to(torch.long)
data = T.ToUndirected()(data)
print(data)

HeteroData(
  user={ node_id=[609] },
  movie={
    node_id=[2429],
    x=[2429, 20],
  },
  director={
    node_id=[1229],
    x=[1229, 2],
  },
  (user, rates, movie)={
    edge_index=[2, 55453],
    edge_label=[55453],
  },
  (director, directs, movie)={
    edge_index=[2, 2605],
    edge_label=[2605],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 55453],
    edge_label=[55453],
  },
  (movie, rev_directs, director)={
    edge_index=[2, 2605],
    edge_label=[2605],
  }
)


In [306]:
print(len(unique_director_id))
print(director_stats.shape)
print(movie_feat.shape)
print(len(data['movie'].node_id))

1229
torch.Size([1229, 2])
torch.Size([2429, 20])
2429


In [307]:
print(data['user', 'rates', 'movie'].edge_label)
print(ratings_df['rating'].values)

tensor([4, 4, 5,  ..., 3, 3, 4])
[4.  4.  5.  ... 3.5 3.5 4. ]


In [308]:
transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=True,
    edge_types=[("user", "rates", "movie"), ("director", "directs", "movie")],
    rev_edge_types=[("movie", "rev_rates", "user"), ("movie", "rev_directs", "director")],
)
train_data, val_data, test_data = transform(data)

In [309]:
print(data['user', 'rates', 'movie'].edge_label.unique())
print(torch.from_numpy(ratings_df['rating'].values).to(torch.long).unique())
train_data['user', 'rates', 'movie'].edge_label.unique()

tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])


tensor([0, 1, 2, 3, 4, 5, 6])

In [310]:
from torch_geometric.loader import LinkNeighborLoader
# Example configuration with correct tuple formatting for edge_label_index:
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[30,30,30],
    edge_label_index=(('user', 'rates', 'movie'), train_data['user', 'rates', 'movie'].edge_label_index),  # Correct tuple format
    edge_label=train_data['user', 'rates', 'movie'].edge_label,  # Labels for the edges
    batch_size=128,
    shuffle=True,
    neg_sampling_ratio=2.0  # Example negative sampling ratio
)

val_sample = next(iter(train_loader))
print(val_sample)
print(val_sample['user', 'rates', 'movie'].edge_label)


HeteroData(
  user={
    node_id=[609],
    n_id=[609],
    num_sampled_nodes=[4],
  },
  movie={
    node_id=[1945],
    x=[1945, 20],
    n_id=[1945],
    num_sampled_nodes=[4],
  },
  director={
    node_id=[714],
    x=[714, 2],
    n_id=[714],
    num_sampled_nodes=[4],
  },
  (user, rates, movie)={
    edge_index=[2, 22735],
    edge_label=[384],
    edge_label_index=[2, 384],
    e_id=[22735],
    num_sampled_edges=[3],
    input_id=[128],
  },
  (director, directs, movie)={
    edge_index=[2, 1142],
    edge_label=[1875],
    edge_label_index=[2, 1875],
    e_id=[1142],
    num_sampled_edges=[3],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 13049],
    edge_label=[13049],
    e_id=[13049],
    num_sampled_edges=[3],
  },
  (movie, rev_directs, director)={
    edge_index=[2, 1162],
    edge_label=[1162],
    e_id=[1162],
    num_sampled_edges=[3],
  }
)
tensor([1, 1, 1, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 5, 4, 1, 1, 1, 1, 1, 3, 1, 1, 3,
        1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1

In [323]:
# Define the validation seed edges:
edge_label_index = val_data["user", "rates", "movie"].edge_label_index
edge_label = val_data["user", "rates", "movie"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[30,30,30],
    edge_label_index=(("user", "rates", "movie"), edge_label_index),
    edge_label=edge_label,
    batch_size=64,
    shuffle=False,
    neg_sampling_ratio=2.0,
)
val_sample = next(iter(val_loader))   

In [312]:
val_sample

HeteroData(
  user={
    node_id=[608],
    n_id=[608],
    num_sampled_nodes=[4],
  },
  movie={
    node_id=[1931],
    x=[1931, 20],
    n_id=[1931],
    num_sampled_nodes=[4],
  },
  director={
    node_id=[830],
    x=[830, 2],
    n_id=[830],
    num_sampled_nodes=[4],
  },
  (user, rates, movie)={
    edge_index=[2, 26925],
    edge_label=[64],
    edge_label_index=[2, 64],
    e_id=[26925],
    num_sampled_edges=[3],
    input_id=[64],
  },
  (director, directs, movie)={
    edge_index=[2, 1392],
    edge_label=[780],
    edge_label_index=[2, 780],
    e_id=[1392],
    num_sampled_edges=[3],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 14763],
    edge_label=[14763],
    e_id=[14763],
    num_sampled_edges=[3],
  },
  (movie, rev_directs, director)={
    edge_index=[2, 1286],
    edge_label=[1286],
    e_id=[1286],
    num_sampled_edges=[3],
  }
)

In [320]:
from torch_geometric.nn import GraphConv, HeteroConv, to_hetero
from torch.nn.functional import cross_entropy

num_features = 64
hidden_dim = 64

model = Model(64, 64, 8)
sample = next(iter(train_loader))
output = model(sample)
target = sample['user', 'rates', 'movie'].edge_label
loss = cross_entropy(output, target)
print(loss)

tensor(248.9620, grad_fn=<NllLossBackward0>)


In [321]:
from torch_geometric.nn import GraphConv, to_hetero

class GNN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim):
        super().__init__()
        self.conv1 = GraphConv(num_features, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, hidden_dim)
        self.conv3 = GraphConv(hidden_dim, hidden_dim)
    
    def forward(self, x, edge_index):#, edge_weight):
        x = torch.relu(self.conv1(x, edge_index))#, edge_weight))
        x = torch.relu(self.conv2(x, edge_index))#, edge_weight)
        x = torch.relu(self.conv3(x, edge_index))#, edge_weight)
        return x

class Classifier(torch.nn.Module):
    def __init__(self, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = torch.nn.Linear(2*hidden_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_dim, num_classes)  # Final layer that outputs class scores

    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Extract features for users and movies according to the edge indices
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Concatenate user and movie features
        x = torch.cat((edge_feat_user, edge_feat_movie), dim=1)  # Concatenation along the feature dimension

        # Pass the concatenated vector through the dense layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


class Model(torch.nn.Module):
    # TODO: add data in function header?
    def __init__(self, num_features, hidden_dim, num_classes):
        super().__init__()
        self.movie_lin = torch.nn.Linear(20, hidden_dim) # TODO: thats the number of neighbors?
        self.director_lin = torch.nn.Linear(2, hidden_dim)
        self.user_emb = torch.nn.Embedding(data['user'].num_nodes, hidden_dim)
        self.movie_emb = torch.nn.Embedding(data['movie'].num_nodes, hidden_dim)
        self.director_emb = torch.nn.Embedding(data['director'].num_nodes, hidden_dim)
        self.gnn = GNN(num_features, hidden_dim)
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier(hidden_dim, num_classes)

    def forward(self, data:HeteroData) -> Tensor:
        x_dict = {
            "user": self.user_emb(data['user'].node_id),
            "movie": self.movie_lin(data['movie'].x) + self.movie_emb(data['movie'].node_id),
            "director": self.director_lin(data['director'].x) + self.director_emb(data['director'].node_id) 
        }
        x_dict = self.gnn(x_dict, data.edge_index_dict)#, data.edge_weight_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["movie"],
            data["user", "rates", "movie"].edge_label_index,
        )
        return pred

model = Model(256, 256, 8)
print(model)

Model(
  (movie_lin): Linear(in_features=20, out_features=256, bias=True)
  (director_lin): Linear(in_features=2, out_features=256, bias=True)
  (user_emb): Embedding(609, 256)
  (movie_emb): Embedding(2429, 256)
  (director_emb): Embedding(1229, 256)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__rates__movie): GraphConv(256, 256)
      (director__directs__movie): GraphConv(256, 256)
      (movie__rev_rates__user): GraphConv(256, 256)
      (movie__rev_directs__director): GraphConv(256, 256)
    )
    (conv2): ModuleDict(
      (user__rates__movie): GraphConv(256, 256)
      (director__directs__movie): GraphConv(256, 256)
      (movie__rev_rates__user): GraphConv(256, 256)
      (movie__rev_directs__director): GraphConv(256, 256)
    )
    (conv3): ModuleDict(
      (user__rates__movie): GraphConv(256, 256)
      (director__directs__movie): GraphConv(256, 256)
      (movie__rev_rates__user): GraphConv(256, 256)
      (movie__rev_directs__director): GraphConv(256, 256)
   

In [325]:
import tqdm
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

#model = torch.load("./model/auc_8092.pt")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
save = False


for epoch in range(50):
    total_loss = 0
    total_examples = 0
    model.train()
    for train_sample in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        train_sample.to(device)
        pred = model(train_sample)
        label = train_sample['user', 'rates', 'movie'].edge_label
        loss = F.cross_entropy(pred, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * label.size(0) # TODO: numel?!
        total_examples += label.size(0)

    preds = []
    ground_truths = []
    model.eval()
    for val_sample in tqdm.tqdm(val_loader):
        with torch.no_grad():
            val_sample.to(device)
            preds.append(model(val_sample))
            ground_truths.append(val_sample["user", "rates", "movie"].edge_label)
    pred = torch.softmax(torch.cat(preds, dim=0), dim=1).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    auc = roc_auc_score(ground_truth, pred, multi_class='ovo')
    if epoch % 1 == 0: 
        print(f'Epoch: {epoch:03d}, Loss:{total_loss/total_examples:.4f}')
        print(f"Validation AUC: {auc:.4f}")

if save: torch.save(model, f"./model/auc_{int(auc*10000)}.pt")

Device: cuda


100%|████████████████████████████████████████| 260/260 [00:01<00:00, 164.74it/s]


Epoch: 000, Loss:0.8157
Validation AUC: 0.7076


100%|████████████████████████████████████████| 260/260 [00:00<00:00, 341.34it/s]


Epoch: 001, Loss:0.8121
Validation AUC: 0.7114


...

Epoch: 031, Loss:0.7546
Validation AUC: 0.7155


100%|████████████████████████████████████████| 260/260 [00:00<00:00, 335.99it/s]


Epoch: 032, Loss:0.7520
Validation AUC: 0.7164


 98%|███████████████████████████████████████▎| 307/312 [00:01<00:00, 182.19it/s]


KeyboardInterrupt: 